In [1]:
from hsmm_core.data_utils import DataLoader, TradingHours
from hsmm_core.feature_spaces import hmm_features
from hsmm_core.hsmm_runner import HmmCalibration
import time
from hsmm_core.consts import InitialisationMethod
from hsmm_core.data_utils import TradingHours, DataLoader
from hsmm_core.labelling import DataLabellingSimple
from hsmm_core.consts import ThresholdMethod, LabellingChoice
import pandas as pd




In [2]:
import os
os.getenv('FINANCE_DATA')
ticker= 'SYNT_2states'
synt_data=os.listdir(os.path.join(os.getenv('FINANCE_DATA'), ticker))
ticker_dir = os.path.join(os.getenv('FINANCE_DATA'),ticker)
test_file_name =os.path.join(ticker_dir,synt_data[0])


In [3]:
###sort out the data###
data_symbols=os.listdir(os.getenv('FINANCE_DATA'))
synt_data_files = os.listdir(os.path.join(os.getenv('FINANCE_DATA'),'SYNT_2states'))
synt_data_dates=[os.path.splitext(file_name)[0] for file_name in synt_data_files]
synt_data_dates.sort()#sort the dates. no special reason makes it cleaner to play with the indexation

In [5]:
n_hidden_states = 2

data_loader_init = {
    'trading_hours_filter': TradingHours.only_mkt_hours,
}


init_params = {
    'obs_model_name': 'ExpIndMixDiracGauss',
    'em_obs_init_method': InitialisationMethod.cluster,
    'em_hidden_init_method': InitialisationMethod.uniform,
    'no_hidden_states': n_hidden_states,
    'update_tag': 'tpsml'
}
sd = synt_data_dates[0]
ed = synt_data_dates[4]

data_loader = DataLoader(**data_loader_init)

data_loader_hash = data_loader.data_loader_hash()

data = data_loader.load_trades_data(ticker, start_date=sd, end_date=ed)

hmm_calibration_engine = HmmCalibration(init_params=init_params)
# hmm_calibration_engine.run_calibration_all_data(ticker, data,  data_loader_hash,
#                                                 force_recalc=True, use_multiprocessing=False,
#                                                 n_processes=2)


In [6]:
def fwd_dates(_dates_list, _key_date):
    # returns a list of dates that are forward from the key_date
    fwd_dates_list = [i for i in _dates_list if i > _key_date]
    return fwd_dates_list

In [7]:
features_engine = hmm_features()

s = time.time()
for date, date_data in data.iteritems():
    stored_hmm, _ = hmm_calibration_engine.get_calibrated_hmm(ticker, date, data_loader_hash)

    if stored_hmm is None:
       raise ValueError("No hmm available for date {}".format(date))

    features_engine.hmm = stored_hmm

    features = features_engine.generate_features(date_data)

In [ ]:
dates_for_features=fwd_dates(_dates_list=synt_data_dates, _key_date=sd)

In [ ]:
for date, date_data in data.iteritems():
    stored_hmm, _ = hmm_calibration_engine.get_calibrated_hmm(ticker, date, data_loader_hash)
    
    features_engine.hmm = stored_hmm
    

    features = features_engine.generate_features(date_data)
 

In [ ]:

#data_for_features = dates_for_features[i],
for index, date in enumerate(synt_data_dates):
    seq = (synt_data_dates[index], "csv"); # This is sequence of strings.
#     print str(".".join( seq ))
    input_date_data = pd.read_csv(os.path.join(ticker_dir, str(".".join(seq))))
    features_oos=features_engine.generate_features(input_date_data)


In [ ]:
seq[0]

In [ ]:
labelling_method_params = [
# {
#     'labelling_method': LabellingChoice.ep_in_window,
#     'rolling_window': 10,
#     'threshold_method': ThresholdMethod.historical_vol_ma,
#     'threshold_vol_window': 5,
#     'updown_thrshd_vol_pct': 10.,
#     'trading_hours_filter': trading_hours_filter,
# },
{
    'labelling_method': LabellingChoice.price_move_in_window,
    'rolling_window': 2,
    # Uncomment below if you want to check a price move only above a certain level
    # 'updown_threshold': 0.1
    # 'threshold_method': ThresholdMethod.arbitrary,
}]


trading_hours_filter = TradingHours.only_mkt_hours

sd= seq[0]
ed=sd

data_loader = DataLoader(trading_hours_filter)

input_date_data = data_loader.load_trades_data(ticker, start_date=sd, end_date=ed)

for label_init in labelling_method_params:
    labeller = DataLabellingSimple(label_init)
    the_label = labeller.get_label_name()
    labeller.label_training_data(input_date_data)

print "ok"

In [ ]:
main_path = os.path.join(os.path.expanduser("~"), 'Data/features_models/')

models_path = os.path.join(main_path, 'models')
ticker_models_path = os.path.join(models_path, ticker)

In [ ]:
#input_date_data[str(seq[0])]
os.path.join(ticker_models_path)